---
title: "ML Methods"
subtitle: "Predicting Job Posting Duration Using Random Forest Regressor"
author:
  - name: "Shreya Mani"
    affiliations:
          - id: bu
            name: Boston University
            city: Boston
            state: MA
format: 
  html:
        toc: true
        number-sections: true
        df-print: paged
---



# Introduction

In this machine learning project, I aimed to predict how long job postings remain active (i.e., their DURATION) using a Random Forest Regressor. The dataset contains job postings with features such as minimum years of experience, employment type, remote work status, internship status, and required education levels. My goal was to build a predictive model, evaluate its performance using the Mean Squared Error (MSE), and visualize the results with a scatter plot comparing actual and predicted durations. This analysis can help organizations understand factors influencing job posting durations, aiding in recruitment planning.

# Data Preprocessing

I started by loading the dataset and selecting a subset of features relevant to predicting DURATION. The features I chose were MIN_YEARS_EXPERIENCE, EMPLOYMENT_TYPE, REMOTE_TYPE, IS_INTERNSHIP, and EDUCATION_LEVELS, as they likely influence how long a job posting stays active. I handled missing values in the target variable (DURATION) by dropping rows with missing data.

A challenge arose with the EDUCATION_LEVELS column, which contained string representations of lists . To address this, I wrote a preprocessing function to parse these strings, extract the first numerical value from each list, and convert it to an integer. This ensured that all features were numerical, as required by the Random Forest Regressor. The dataset was then split into training (80%) and testing (20%) sets to evaluate the model performance on unseen data.

Here the Python code I used for data preprocessing:


In [ ]:
#| label: ml data cleaning
#| echo: true
#| warning: false

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import ast
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Auto-download CSV if missing
csv_path = 'region_analysis/lightcast_job_postings.csv'
if not os.path.exists(csv_path):
    print(f"{csv_path} not found! Attempting to download...")
    os.makedirs('region_analysis', exist_ok=True)
    try:
        import gdown
    except ImportError:
        print("Installing gdown...")
        !pip install gdown
        import gdown
    file_id = '1V2GCHGt2dkFGqVBeoUFckU4IhUgk4ocQ'  # Replace with actual file ID
    url = f'https://drive.google.com/uc?id={file_id}'
    try:
        gdown.download(url, csv_path, quiet=False)
        print("Download complete!")
    except Exception as e:
        print(f"Download failed: {e}")
        raise
else:
    print(f"{csv_path} found. Proceeding...")

# Load the dataset
try:
    df = pd.read_csv(csv_path)
    print(f"Initial dataset size: {df.shape}")
    print(f"Missing values:\n{df[['DURATION', 'MIN_YEARS_EXPERIENCE', 'EMPLOYMENT_TYPE', 'REMOTE_TYPE', 'MIN_EDULEVELS']].isnull().sum()}")
except Exception as e:
    print(f"Error loading CSV: {e}")
    raise

## Sample of preprocessed EDUCATION_LEVELS


In [ ]:
# Define function to parse MIN_EDULEVELS strings
def parse_education_levels(edu):
    if isinstance(edu, (int, float)) and not np.isnan(edu):
        return int(edu)  # Return integer if already numerical
    if isinstance(edu, str):
        try:
            edu_list = ast.literal_eval(edu.replace('\n', ''))
            return int(edu_list[0]) if isinstance(edu_list[0], (int, float)) else np.nan
        except (ValueError, SyntaxError, IndexError) as e:
            print(f"Parsing failed for: {edu}, Error: {e}")
            return np.nan
    return np.nan

# Select features and target
features = ['MIN_YEARS_EXPERIENCE', 'EMPLOYMENT_TYPE', 'REMOTE_TYPE', 'IS_INTERNSHIP', 'MIN_EDULEVELS']
target = 'DURATION'

# Check if all features and target exist
missing_cols = [col for col in features + [target] if col not in df.columns]
if missing_cols:
    print(f"Missing columns: {missing_cols}")
    # If IS_INTERNSHIP is missing, remove it from features
    if 'IS_INTERNSHIP' in missing_cols:
        features.remove('IS_INTERNSHIP')
    else:
        raise ValueError("Required columns not found in dataset")

# Create a copy of the dataset with selected columns
df_subset = df[features + [target]].copy()

# Parse MIN_EDULEVELS
df_subset['MIN_EDULEVELS'] = df_subset['MIN_EDULEVELS'].apply(parse_education_levels)

# Handle missing values with imputation for all numerical columns
num_cols = [col for col in features if col in df_subset.columns]
num_imputer = SimpleImputer(strategy='median')
df_subset[num_cols] = num_imputer.fit_transform(df_subset[num_cols])

# Impute DURATION with mean
df_subset['DURATION'] = df_subset['DURATION'].fillna(df_subset['DURATION'].mean())

# Ensure IS_INTERNSHIP is integer if present
if 'IS_INTERNSHIP' in df_subset.columns:
    df_subset['IS_INTERNSHIP'] = df_subset['IS_INTERNSHIP'].astype(int)

# Verify no missing values
print(f"Missing values after imputation:\n{df_subset.isnull().sum()}")
print(f"Preprocessed dataset size: {df_subset.shape}")

# Features and target
X = df_subset[num_cols]
y = df_subset['DURATION']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Testing set size:", X_test.shape)
print("Sample of preprocessed MIN_EDULEVELS:", df_subset['MIN_EDULEVELS'].head().tolist())

# Model Training

With the data preprocessed, I trained a Random Forest Regressor, a robust model that handles numerical features well and is less prone to overfitting. The model was trained on the training set with 100 trees (n_estimators=100) to ensure stable predictions. Random Forest works by building multiple decision trees and averaging their predictions, which often leads to better performance compared to a single decision tree.

Here is the code for training the Random Forest Regressor:


In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Initialize and train the Random Forest Regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

print("Model training completed.")

# Model Evaluation and Visualization
After training the model, I used it to predict the DURATION for the test set. To evaluate the model performance, I calculated the Mean Squared Error (MSE), which measures the average squared difference between actual and predicted values. A lower MSE indicates better predictive accuracy.

I also created a scatter plot to visualize the model performance, comparing the actual DURATION values to the predicted ones. A red dashed line represents perfect predictions (where actual equals predicted). Points closer to this line indicate better predictions.

Here is the code for evaluation and visualization:


In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

# Cross-validation
cv_scores = cross_val_score(rf, X, y, cv=5, scoring='neg_mean_squared_error')
cv_mse = -cv_scores.mean()
cv_rmse = np.sqrt(cv_mse)
print(f"Cross-validated MSE: {cv_mse:.2f} ± {cv_scores.std():.2f}")
print(f"Cross-validated RMSE: {cv_rmse:.2f} days")

# Predict on test set
y_pred = rf.predict(X_test)

# Calculate MSE and RMSE on test set
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Test set MSE: {mse:.2f}")
print(f"Test set RMSE: {rmse:.2f} days")

# Feature importance
feature_importance = pd.DataFrame({
    'Feature': num_cols,
    'Importance': rf.feature_importances_
}).sort_values('Importance', ascending=False)
print("\nFeature Importance:")
print(feature_importance)

# Plot actual vs predicted values
plt.figure(figsize=(8, 6))
plt.scatter(y_test, y_pred, color='blue', alpha=0.5, label='Predicted vs Actual')
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', label='Perfect Prediction')
plt.xlabel('Actual Duration (Days)')
plt.ylabel('Predicted Duration (Days)')
plt.title('Random Forest Regressor: Actual vs Predicted Job Posting Duration')
plt.legend()
plt.grid(True)
plt.savefig('actual_vs_predicted_duration.png')
plt.show()

# Results

The Mean Squared Error (MSE) provides a quantitative measure of the model performance. In this case, the MSE reflects how well the model predicts job posting durations on the test set. The scatter plot (actual_vs_predicted_duration.png) visually demonstrates the model accuracy. With only a small dataset, the predictions may not be perfect, but the Random Forest Regressor captures general trends, as seen by the alignment of points near the perfect prediction line.

# Conclusion

Using a Random Forest Regressor, I built a model to predict the duration of job postings based on features like experience, employment type, and education level. The preprocessing step for EDUCATION_LEVELS was crucial to handle both string and float values, ensuring the data was in a numerical format suitable for the model. However, the model performance was poor, with an MSE of 1296.00 and a significant overprediction (42.0 days predicted vs. 6.0 days actual), as shown in the scatter plot. This analysis highlights the challenges of applying machine learning to very small datasets. Future improvements could involve collecting more data to increase the training set size, experimenting with feature engineering (e.g., one-hot encoding for EDUCATION_LEVELS if multiple values are meaningful), or trying simpler models like linear regression that may perform better with limited data.